In [1]:
import pandas as pd
import numpy as np
# import string as s
import seaborn as sns
import sz_toolbox as sz
# import math
# import tables
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style, colors
# from itertools import groupby
import matplotlib.patches as patches
from matplotlib import gridspec
import scipy.stats as stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata, leaves_list
from scipy.spatial.distance import pdist
# from scipy.ndimage.interpolation import shift
# matplotlib.style.use('ggplot')
# %matplotlib inline
# matplotlib.rcParams['figure.figsize'] = (8, 6)
# matplotlib.rcParams['figure.facecolor'] = 'white'
# import mpl_toolkits.mplot3d.axes3d as p3
# from sklearn.cluster import AgglomerativeClustering
# from sklearn.datasets.samples_generator import make_swiss_roll
# matplotlib.style.use('ggplot')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Overview

 53 samples, need to remove 2 samples: 'HTMCP-03-06-02007', and 'HTMCP-03-06-02026'
 
 

In [56]:
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients.txt'
patients = pd.read_csv(f1, header=None)[0].values.tolist()
assert len(patients) == 118
patients[:3]
'HTMCP-03-06-02007' in patients
'HTMCP-03-06-02026' in patients

['HTMCP-03-06-02001', 'HTMCP-03-06-02002', 'HTMCP-03-06-02003']

False

False

# 2. constants for notebook

In [59]:
wkdir = '/projects/da_workspace/DA-167_Cervical_epi_clustering/'
colormap_file = '/projects/trans_scratch/validations/workspace/szong/Cervical/color_maps.txt'
mark = 'H3K4me3'
# mark = 'H3K27ac'
# mark = 'H3K4me1'
# need to defined based on what make sense
overlap = 1

# 3. peak files 

In [64]:
%%bash -s $wkdir
wkdir=$1
cd $wkdir
for mark in 'H3K4me3' 'H3K27ac' 'H3K4me1';
do 
    echo "working on $mark"
    ls -1 /projects/chipseq/macs/A*/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/*/*$mark*Peak|grep -v gapped > a.tmp
    cat a.tmp |awk -F "/" '{print $5"\t"$0}'|awk -F "_" '{print $3"\t"$0}'>d.tmp
    while read a b c;do grep $b 57_chipseq_samples_library_ids.csv|cut -f 1|\
    awk -v lib=$b -v path=$c -v mark=$a -F "-01" '{print $1"\t"lib"\t"mark"\t"path}' ;done \
    < d.tmp |sort -k1 -k2|grep -v 'HTMCP-03-06-02007'|grep -v 'HTMCP-03-06-02026' > $mark"_peak_file_path.txt"
    rm a.tmp  d.tmp
    head -2 $mark"_peak_file_path.txt"
    wc -l $mark"_peak_file_path.txt"
done


working on H3K4me3
HTMCP-03-06-02001	A84611	H3K4me3	/projects/chipseq/macs/A84611/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/369/A84611_H3K4me3_peaks.narrowPeak
HTMCP-03-06-02002	A94512	H3K4me3	/projects/chipseq/macs/A94512/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/687/A94512_H3K4me3_peaks.narrowPeak
51 H3K4me3_peak_file_path.txt
working on H3K27ac
HTMCP-03-06-02001	A84615	H3K27ac	/projects/chipseq/macs/A84615/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/370/A84615_H3K27ac_peaks.narrowPeak
HTMCP-03-06-02002	A94516	H3K27ac	/projects/chipseq/macs/A94516/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/691/A94516_H3K27ac_peaks.narrowPeak
52 H3K27ac_peak_file_path.txt
working on H3K4me1
HTMCP-03-06-02001	A84610	H3K4me1	/projects/chipseq/macs/A84610/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/372/A84610_H3K4me1_peaks.broadPeak
HTMCP-03-06-02002	A94511	H3K4me1	/projects/chipseq/macs/A94511/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/686/A94511_H3K4me1_peaks.broadPe

# 4. merge peaks from all patients

In [71]:
%%bash -s $wkdir $overlap
wkdir=$1
overlap=$2
cd $wkdir
promoters=/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/hg19v69_genes.TSS_2000.pc.bed.cleaned
bedtools=/home/rcorbett/aligners/bedtools/BEDTools-Version-2.15.0/bin/bedtools
hotspots=/projects/trans_scratch/validations/workspace/szong/Cervical/hotspots/rainstorm/rainstorm_hotspots_with_qvalue.txt
# for mark in 'H3K4me3' 'H3K27ac' 'H3K4me1';
# do 
#     echo "working on $mark"
#     # merge all broad peak regions from all patients
#     while read a b c d; do cut -f 1-3 $d|grep -v "^GL"|grep -v "^MT"|sed 's/^X/23/g'|sed 's/^Y/24/g';done <$mark"_peak_file_path.txt"|sort -k1n -k2n -k3n >$mark.peaks.txt
#     $bedtools merge -i $mark.peaks.txt >$mark.peaks.merge.txt
#     rm $mark.peaks.txt
# done

# # intersect H3K4me1 and H3K27ac, H3K4me3 and H3K27ac
# $bedtools intersect -a H3K27ac.peaks.merge.txt -b H3K4me1.peaks.merge.txt -f 1 > H3K27ac_H3K4me1_intersection.txt
# $bedtools intersect -a H3K27ac.peaks.merge.txt -b H3K4me3.peaks.merge.txt -f 0.5 > H3K27ac_H3K4me3_intersection.txt

# $bedtools intersect -a $hotspots.tmp -b H3K27ac_H3K4me1_intersection.txt -f 0.3 -wa|cut -f 4- > $hotspots.H3K27ac_H3K4me1.intersected.txt
$bedtools intersect -a $hotspots.tmp -b H3K27ac_H3K4me3_intersection.txt -f 0.3 -wa|cut -f 4- > $hotspots.H3K27ac_H3K4me3.intersected.txt

In [40]:
adf[0].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23])

In [67]:
adf = pd.read_csv(f'{wkdir}H3K27ac_H3K4me1_intersection.txt', sep='\t', header=None)
adf = adf[adf[0].isin([i for i in range(1,24)]+['X', 'Y'])]
adf.head()
(adf[2] - adf[1]).sum()

,0,1,2
0,1,11607,11933
1,1,12894,14501
2,1,20507,20785
3,1,21131,21532
4,1,21600,22656


505723890

In [68]:
adf = pd.read_csv(f'{wkdir}H3K27ac_H3K4me3_intersection.txt', sep='\t', header=None)
adf = adf[adf[0].isin([i for i in range(1,24)]+['X', 'Y'])]
adf.head()
(adf[2] - adf[1]).sum()

,0,1,2
0,1,9973,10451
1,1,13397,14492
2,1,26448,26594
3,1,27858,31435
4,1,32758,33410


230330804

In [69]:
%%bash -s $wkdir $mark $overlap
wkdir=$1
mark=$2
overlap=$3
# intersect hotspots with H3K4me1 peaks
bedtools=/home/rcorbett/aligners/bedtools/BEDTools-Version-2.15.0/bin/bedtools
hotspots='/projects/trans_scratch/validations/workspace/szong/Cervical/hotspots/rainstorm/rainstorm_hotspots_with_qvalue.txt'
# awk '{print $2"\t"$3"\t"$4"\t"$0}' $hotspots|grep -v 'mostFreqVClassification' >$hotspots.tmp
$bedtools intersect -a $hotspots.tmp -b H3K27ac_H3K4me1_intersection.txt -f 0.3 -wa|cut -f 4- > $hotspots.H3K27ac_H3K4me1.intersected.txt

Error: The requested bed file (H3K27ac_H3K4me1_intersection.txt) could not be opened. Exiting!
